In [64]:
!wget -nv -O kesci_submit https://cdn.kesci.com/submit_tool/v1/kesci_submit&&chmod +x kesci_submit

In [1]:
import pandas as pd
import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import stats
import lightgbm as lgb
%matplotlib inline
import gc
import lightgbm as lgb
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

# Memory optimize

In [2]:
%%time
register = pd.read_table('/mnt/datasets/fusai/user_register_log.txt', sep='\t', header=None, 
             names=['user_id', 'register_day',
                    'register_type', 'device_type']).sort_values(
                                            by=['user_id', 'register_day'])
register =  register.apply(pd.to_numeric, downcast='unsigned')
launch = pd.read_table('/mnt/datasets/fusai/app_launch_log.txt', sep='\t', header=None,
                      names=['user_id', 'launch_day']).sort_values(by=['user_id',
                                                                     'launch_day'])
launch =  launch.apply(pd.to_numeric, downcast='unsigned')
video = pd.read_table('/mnt/datasets/fusai/video_create_log.txt', sep='\t', header=None,
                     names=['user_id', 'create_day']).sort_values(by=['user_id',
                                                                     'create_day'])
video =  video.apply(pd.to_numeric, downcast='unsigned')
activity = pd.read_table('/mnt/datasets/fusai/user_activity_log.txt', sep='\t', header=None,
                        names=['user_id', 'act_day', 'act_page', 'video_id', 'author_id',
                              'act_type']).sort_values(by=['user_id', 'act_day'])
activity =  activity.apply(pd.to_numeric, downcast='unsigned')
register.reset_index(drop=True).to_pickle('/home/kesci/input/register.pkl',)
launch.reset_index(drop=True).to_pickle('/home/kesci/input/launch.pkl',)
video.reset_index(drop=True).to_pickle('/home/kesci/input/create.pkl',)
activity.reset_index(drop=True).to_pickle('/home/kesci/input/activity.pkl')
del register, launch, video, activity
gc.collect()

CPU times: user 3min 16s, sys: 46.5 s, total: 4min 3s
Wall time: 4min 3s


In [ ]:
!tar -jcvf <要生成的压缩文件名> <要压缩的文件> # 压缩
!tar -jxvf <要解压缩的文件> # 解压缩

# load preprocess raw data 

In [2]:
%%time
register = pd.read_pickle('/home/kesci/input/register.pkl',)
launch = pd.read_pickle('/home/kesci/input/launch.pkl',)
create = pd.read_pickle('/home/kesci/input/create.pkl',)
activity = pd.read_pickle('/home/kesci/input/activity.pkl')

CPU times: user 616 ms, sys: 1.99 s, total: 2.6 s
Wall time: 2.61 s


In [12]:
print('register user:', len(register['user_id']),
      'launch user:', len(np.unique(launch['user_id'])),
      'create user:', len(np.unique(create['user_id'])),
      'activity user:', len(np.unique(activity['user_id'])),)

register user: 572132 launch user: 572132 create user: 83366 activity user: 482781


# 划分数据集

In [4]:
len(register)

572132

In [5]:
len(np.unique(register['user_id']))

572132

In [7]:
def get_new_label(register, create, launch, activity, s_day_user, e_day_user, s_day_fea, e_day_fea, *label_cut_day):
    train_register = register[(s_day_user <= register['register_day']) & 
         (register['register_day'] <= e_day_user)].reset_index(drop=True)
    train_create = create[(s_day_fea <= create['create_day']) & 
         (create['create_day'] <= e_day_fea)].reset_index(drop=True)
    train_launch = launch[(s_day_fea <= launch['launch_day']) & 
         (launch['launch_day'] <= e_day_fea)].reset_index(drop=True)
    train_activity = activity[(s_day_fea <= activity['act_day']) & 
         (activity['act_day'] <= e_day_fea)].reset_index(drop=True)
    if len(label_cut_day) == 2:
        s_day_label = label_cut_day[0]
        e_day_label = label_cut_day[1]
        test_create = create[(s_day_label <= create['create_day']) & 
             (create['create_day'] <= e_day_label)]
        test_launch = launch[(s_day_label <= launch['launch_day']) & 
             (launch['launch_day'] <= e_day_label)]
        test_activity = activity[(s_day_label <= activity['act_day']) & 
             (activity['act_day'] <= e_day_label)]
        #test_cre_id =   test_create.user_id.values.tolist() 
        #test_lau_id =   test_launch.user_id.values.tolist() 
        #test_act_id =   test_activity.user_id.values.tolist()
        #all_id = set(test_cre_id + test_lau_id + test_act_id)
        
        cre_reg = pd.merge(train_register, test_create, how='left', on='user_id') 
        train_register['fir_cre_day2final'] = cre_reg.groupby('user_id').apply(lambda x : x['create_day'].iloc[0]- e_day_fea).values
        train_register['fir_cre_day2final'] = train_register['fir_cre_day2final'].fillna(e_day_label+1-e_day_fea)
        print('1 done')
        
        lau_reg = pd.merge(train_register, test_launch, how='left', on='user_id') 
        train_register['fir_lau_day2final'] = lau_reg.groupby('user_id').apply(lambda x : x['launch_day'].iloc[0]- e_day_fea).values
        train_register['fir_lau_day2final'] = train_register['fir_lau_day2final'].fillna(e_day_label+1-e_day_fea)
        print('2 done')
        
        act_reg = pd.merge(train_register, test_activity, how='left', on='user_id')
        train_register['fir_act_day2final'] = act_reg.groupby('user_id').apply(lambda x : x['act_day'].iloc[0]- e_day_fea).values
        train_register['fir_act_day2final'] = train_register['fir_act_day2final'].fillna(e_day_label+1-e_day_fea)
        print('3 done')
        
        train_register['future_num'] = act_reg.groupby('user_id').apply(lambda x : x['act_day'].notnull().sum()).values +\
                                     lau_reg.groupby('user_id').apply(lambda x : x['launch_day'].notnull().sum()).values +\
                                     cre_reg.groupby('user_id').apply(lambda x : x['create_day'].notnull().sum()).values 
        #train_register['future_num'] = [( len(test_create[test_create['user_id'] == i]) + \
        #                     len(test_launch[test_launch['user_id'] == i]) + \
        #                     len(test_activity[test_activity['user_id'] == i]) ) if i in all_id else 0\
        #                     for i in train_register.user_id]
        print('4 done')
    return train_register[['user_id', 'fir_cre_day2final', 'fir_lau_day2final', 'fir_act_day2final', 'future_num']]      

In [8]:
%%time
train1_new_label = get_new_label(register, create, launch, activity,1, 16, 1, 16, 17, 23)
train2_new_label = get_new_label(register, create, launch, activity,1, 23, 8, 23, 24, 30)

1 done
2 done
3 done
4 done
1 done
2 done
3 done
4 done
CPU times: user 12min 8s, sys: 28.3 s, total: 12min 37s
Wall time: 12min 38s


In [148]:
train1_new_label.to_hdf('new_label.h5', key='train1_new_label', complevel=9, complib='zlib')
train2_new_label.to_hdf('new_label.h5', key='train2_new_label', complevel=9, complib='zlib')